<a href="https://colab.research.google.com/github/ruslanmv/TensorflowTTS/blob/master/TensorflowTTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet gradio
!git clone https://github.com/TensorSpeech/TensorFlowTTS.git
!cd TensorFlowTTS
!pip --quiet install /content/TensorFlowTTS/
!pip --quiet install git+https://github.com/repodiac/german_transliterate
!pip --quiet install SoundFile


     |████████████████████████████████| 3.4 MB 4.2 MB/s 
     |████████████████████████████████| 1.9 MB 36.4 MB/s 
     |████████████████████████████████| 206 kB 48.5 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
     |████████████████████████████████| 3.2 MB 35.2 MB/s 
     |████████████████████████████████| 961 kB 48.0 MB/s 
Cloning into 'TensorFlowTTS'...
remote: Enumerating objects: 10512, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 10512 (delta 115), reused 148 (delta 95), pack-reused 10290
Receiving objects: 100% (10512/10512), 133.26 MiB | 25.56 MiB/s, done.
Resolving deltas: 100% (5055/5055), done.
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for th

In [2]:
import numpy as np
import soundfile as sf
import yaml
import tensorflow as tf
from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoProcessor
import gradio as gr

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [3]:
# initialize fastspeech2 model.
fastspeech2 = TFAutoModel.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")
# initialize mb_melgan model
mb_melgan = TFAutoModel.from_pretrained("tensorspeech/tts-mb_melgan-ljspeech-en")
# inference
processor = AutoProcessor.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")

Downloading:   0%|          | 0.00/125M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.83k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.57k [00:00<?, ?B/s]

In [4]:
def inference(text):
      input_ids = processor.text_to_sequence(text)
      # fastspeech inference

      mel_before, mel_after, duration_outputs, _, _ = fastspeech2.inference(
          input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
          speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
          speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
          f0_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
          energy_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
      )

      # melgan inference
      audio_before = mb_melgan.inference(mel_before)[0, :, 0]
      audio_after = mb_melgan.inference(mel_after)[0, :, 0]

      # save to file
      sf.write('./audio_before.wav', audio_before, 22050, "PCM_16")
      sf.write('./audio_after.wav', audio_after, 22050, "PCM_16")
      return './audio_after.wav'


In [5]:
inputs = gr.inputs.Textbox(lines=5, label="Input Text")
outputs =  gr.outputs.Audio(type="file", label="Output Audio")
title = "Tensorflow TTS"
description = "Gradio demo for TensorFlowTTS: Real-Time State-of-the-art Speech Synthesis for Tensorflow 2. To use it, simply add your text, or click one of the examples to load them."
article = "<p style='text-align: center'><a href='https://ruslanmv.com/'> Check out more Machine Learning projects at my blog </a> | <a href='https://github.com/ruslanmv/TensorFlowTTS'>Github Repo</a></p>"

examples = [
 ["TensorFlowTTS provides real-time state-of-the-art speech synthesis architectures such as Tacotron-2, Melgan, Multiband-Melgan, FastSpeech, FastSpeech2 based-on TensorFlow 2."],
 ["With Tensorflow 2, we can speed-up training/inference progress, optimizer further by using fake-quantize aware and pruning, make TTS models can be run faster than real-time and be able to deploy on mobile devices or embedded systems."]   
]

In [6]:
gr.Interface(inference, inputs, outputs, title=title, description=description, article=article, examples=examples).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://55790.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://55790.gradio.app')